TODO Luki format??

Co prioritizovat? https://github.com/zdenkas/human-tafazzin-variants-database/blob/main/TODO.txt
Konzistence cisel (pozic) a nukleotidu
Jake k tomu pouzit data?
Kolik nekonzistenci tam najdeme? Pak to poslat BSF


In [ ]:
import pandas as pd
from Bio import SeqIO

In [ ]:
import helpers

In [ ]:
database_folder = '../database_versions'

In [ ]:
input_path_prefix = '../database_versions/0001_2023-10-30-21-51-49-628455_Human-TAFAZZIN-Variants-Database_' 
# TODO get this automatically in next notebooks

output_path_prefix = helpers.create_database_output_path_prefix()

In [ ]:
taz_sequences_path = '../data_external/Homo_sapiens_ENST00000601016_6_sequence.fa'
taz_positions_path = '../data_external/ExonsSpreadsheet-Homo_sapiens_Transcript_Exons_ENST00000601016_whole_table.csv'

### Load positions

TAZ positions from ensembl.org in hg38. 

In [ ]:
taz_positions = pd.read_csv(taz_positions_path)

In [ ]:
taz_positions['Start'] = pd.to_numeric(taz_positions['Start'].str.replace(',', ''), errors='coerce')
taz_positions['Start'] = taz_positions['Start'].fillna(0).astype(int)

In [ ]:
taz_positions['End'] = pd.to_numeric(taz_positions['End'].str.replace(',', ''), errors='coerce')
taz_positions['End'] = taz_positions['End'].fillna(0).astype(int)

#### Sanity check for length of exon:

In [ ]:
taz_positions_exons_only = taz_positions[(taz_positions['No.'] != ' ') & (~taz_positions['No.'].isna())].copy(deep=True)
taz_positions_exons_only

In [ ]:
154411852 - 154411539

In [ ]:
taz_positions_exons_only['Length'] = taz_positions_exons_only['Length'].astype(int)

In [ ]:
taz_positions_exons_only['Length'].sum()

### Load TAZ sequences

from ensembl.org

In [ ]:
for seq_record in SeqIO.parse(taz_sequences_path, "fasta"):
    #print(seq_record.id)
    #print(seq_record.name)
    print(seq_record.description)
    #print(repr(seq_record.seq))
    print(len(seq_record))
    print('-----')

In [ ]:
len(seq_record)

In [ ]:
seq_record.seq

In [ ]:
seq_record.seq[-1]

In [ ]:
seq_record

### Load Tafazzin database

In [ ]:
df_pathogenic = pd.read_csv(input_path_prefix + 'pathogenic.csv')
df_vus = pd.read_csv(input_path_prefix + 'vus.csv')
df_exon5 = pd.read_csv(input_path_prefix + 'exon5.csv')
df_benign = pd.read_csv(input_path_prefix + 'benign.csv')

In [ ]:
df_pathogenic.head(1)

In [ ]:
loc_38_col = 'Location in Genome release 38 (hg38)'
loc_37_col = 'Location in Genome release 37 (hg19)'
dna_modif_col = 'DNA Modifications'

In [ ]:
def get_row_from_taz_positions(taz_database_38_loc, taz_positions=taz_positions):
    matching_rows = taz_positions[(taz_positions['Start'] <= taz_database_38_loc) & (taz_positions['End'] >= taz_database_38_loc)]
    #rows = #matching_rows['Exon / Intron'].values
    # If you expect only one match and want to get a single value
    if len(matching_rows) == 1:
        return matching_rows.iloc[0]
    else:
        print("Multiple or no matching rows found!")
        return None

In [ ]:
def find_seq_record(search_string, taz_sequences_path=taz_sequences_path):
    # Use a generator expression to find the first matching seq_record
    seq_record = next((record for record in SeqIO.parse(taz_sequences_path, "fasta") if search_string in record.description), None)
    return seq_record


In [ ]:
taz_positions

In [ ]:
i = 0
for key, row in df_pathogenic.iterrows():
    i += 1
    try:
        taz_database_38_loc = int(row[loc_38_col].split(':')[1])
        taz_database_37_loc = int(row[loc_37_col].split(':')[1])
        taz_database_dna_motif = row[dna_modif_col]
        row_from_taz_positions = get_row_from_taz_positions(taz_database_38_loc)
        exon_intron_from_taz_positions = row_from_taz_positions['Exon / Intron']
        start_from_taz_positions = row_from_taz_positions['Start']
        end_from_taz_positions = row_from_taz_positions['End']
        seq_from_taz_positions = row_from_taz_positions.Sequence.split('}')[-1].strip()
        
        seq_record = find_seq_record(exon_intron_from_taz_positions)

        if seq_record:
            #print(f"Found record: {seq_record.description}")
            sequence = seq_record.seq
            pos_in_sequence = taz_database_38_loc - start_from_taz_positions
            expected_letter = sequence[pos_in_sequence]
            print(taz_database_dna_motif, expected_letter)
            #if not taz_database_dna_motif.endswith(expected_letter):
            #    print('PROBLEM endswith')
            #break
            if i ==5:
                break
            
            
            # TODO hg37, corner cases??
        else:
            print("No matching record found.")
        #break
        
    except Exception as ex:
        pass
        # TODO!! solve these !!print(key, row[loc_38_col]) #, ex)
    #break
        

In [ ]:
154411894 -  154411539

In [ ]:
pos_in_sequence

In [ ]:
seq_record

In [ ]:
seq_record s

In [ ]:
seq_record[51]

In [ ]:
pos_in_sequence

In [ ]:
len(seq_record)

In [ ]:
154411952 - 154411539

In [ ]:
row_from_taz_positions

In [ ]:
str(seq_record.seq)

In [ ]:
seq_record.seq[51]

In [ ]:
for seq_record in SeqIO.parse(taz_sequences_path, "fasta"):
    break

In [ ]:
seq_record[54]

In [ ]:
len()

In [ ]:
seq_from_taz_positions == str(sequence)

In [ ]:
row_from_taz

In [ ]:
seq_from_taz_positions[53]

In [ ]:
row_from_taz_positions

In [ ]:
len(row_from_taz_positions.Sequence.split('}')[-1].strip())

In [ ]:
start_from_taz_positions

In [ ]:
pos_in_sequence

In [ ]:
taz_database_38_loc

In [ ]:
154411539 + 

In [ ]:
end_from_taz_positions

In [ ]:
row_from_taz_positions

TODO dna modifications?? co je 51??

In [ ]:
start_from_taz_positions

In [ ]:
taz_database_38_loc

In [ ]:
pos_in_sequence

In [ ]:
pos_in_sequence

In [ ]:
pos_in_sequence

In [ ]:
assert taz_database_dna_motif.endswith(expected_letter)

In [ ]:
taz_database_dna_motife

In [ ]:
taz_database_dna_motif.endswith(expected_letter)

In [ ]:
expected_letter

In [ ]:
taz_database_dna_motif.endswith(expected_letter)

In [ ]:
expected_letter

In [ ]:
taz_database_dna_motif

In [ ]:
taz_database_dna_modif

In [ ]:
taz_database_dna_modif.endswith(expected_letter)

In [ ]:
expected_letter

In [ ]:
pos_in_sequence

In [ ]:
taz_database_38_loc

In [ ]:
exon_intron_from_taz_positions

In [ ]:
seq_record

In [ ]:
row_from_taz_positions

In [ ]:
exon_intron_from_taz_positions

In [ ]:
start_from_taz_positions

In [ ]:
end_from_taz_positions

In [ ]:
expected_letter

In [ ]:
taz_database_dna_modif

In [ ]:
sequence

In [ ]:
sequence[start_from_taz_positions - start_from_taz_positions]

In [ ]:
row_from_taz_positions

In [ ]:
exon_intron_from_taz_positions

In [ ]:
seq_record.seq

In [ ]:
matching_record.seq

In [ ]:
taz_positions

In [ ]:
taz_positions.Start

In [ ]:
taz_positions.head(20)

In [ ]:
taz_database_38

In [ ]:
taz_database_37

In [ ]:
taz_database_dna_modif

In [ ]:
df_pathogenic.Location.value_counts()